In [48]:
import pandas as pd
import time
import dateparser
import pytz
import argparse
from datetime import datetime
from binance.client import Client
import os
from binance.exceptions import BinanceAPIException

In [49]:
api_key = "iZHsmlsCReb9S6zVO05Vxy8ONQYK8J3CfshgNiRh3HlRShPULMj8EYBClftHBqi1"
api_secret = "4IHk54oeSmmoXGQqWNgi24SJ1uHaTSEBfN48nOhYex8ATFFOj2WoWZQfDFD0pzu1"

In [50]:
client = Client(api_key, api_secret)
tz = pytz.timezone('UTC')
buy = {}
loss = []
quantity = {}

In [51]:
def date_to_milliseconds(date_str):
    # date_str format: "January 01, 2018", "11 hours ago UTC", "now UTC"
    epoch = datetime.utcfromtimestamp(0).replace(tzinfo=pytz.utc)
    d = dateparser.parse(date_str)
    if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
        d = d.replace(tzinfo=pytz.utc)
    return int((d - epoch).total_seconds() * 1000.0)

In [52]:
def interval_to_milliseconds(interval):
    # interval format: 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 1d, 3d, 1w
    ms = None
    seconds_per_unit = {
        "m": 60,
        "h": 60 * 60,
        "d": 24 * 60 * 60,
        "w": 7 * 24 * 60 * 60
    }
    unit = interval[-1]
    if unit in seconds_per_unit:
        try:
            ms = int(interval[:-1]) * seconds_per_unit[unit] * 1000
        except ValueError:
            pass
    return ms

In [53]:
def get_historical_klines(symbol, interval, start_str, end_str=None):
    """Get Historical Klines from Binance
    If using offset strings for dates add "UTC" to date string e.g. "now UTC", "11 hours ago UTC", "1 Dec, 2017"
    :param symbol: Name of symbol pair e.g BNBBTC
    :param interval: Biannce Kline interval
    :param start_str: Start date string in UTC format
    :param end_str: optional - end date string in UTC format
    :return: list of Open High Low Close Volume values
    """
    output_data = []
    limit = 50
    timeframe = interval_to_milliseconds(interval)
    start_ts = date_to_milliseconds(start_str)
    end_ts = None
    if end_str:
        end_ts = date_to_milliseconds(end_str)

    idx = 0
    # it can be difficult to know when a symbol was listed on Binance so allow start time to be before list date
    symbol_existed = False
    while True:
        temp_data = client.get_klines(
            symbol=symbol,
            interval=interval,
            limit=limit,
            startTime=start_ts,
            endTime=end_ts
        )
        # handle the case where our start date is before the symbol pair listed on Binance
        if not symbol_existed and len(temp_data):
            symbol_existed = True
        if symbol_existed:
            output_data += temp_data
            start_ts = temp_data[len(temp_data) - 1][0] + timeframe
        else:
            start_ts += timeframe
        idx += 1
        if len(temp_data) < limit:
            break
        # sleep after every 3rd call to be kind to the API
        if idx % 3 == 0:
            time.sleep(1)

    return output_data


In [54]:
def get_historic_klines(symbol, start, end, interval):
    klines = get_historical_klines(symbol, interval, start, end)
    #print("Klines")
    #print(klines[-1])
    ochl = []
    for kline in klines:
        time1 = int(kline[0])
        open1 = float(kline[1])
        high = float(kline[2])
        low = float(kline[3])
        close = float(kline[4])
        volume = float(kline[5])
        ochl.append([time1, open1, close, high, low, volume])
    #df = pd.DataFrame(ochl)
    #print(symbol)
    #print(df)
    #print(ochl[-1])
    '''
    fig, ax = plt.subplots()
    mpl_finance.candlestick_ochl(ax, ochl, width=1)
    ax.set(xlabel='Date', ylabel='Price', title='{} {}-{}'.format(symbol, start, end))
    plt.show(block=False)
    plt.pause(3)
    plt.close()
    '''
    return ochl


In [75]:
def Main():
    list_of_symbols = ['XMRUSDT', 'LINKUSDT', 'LTCUSDT', 'BNBUSDT', 'EOSUSDT', 'BTCUSDT', 'ETHUSDT', 'XRPUSDT']  # Symbols to be traded
    quantity_1 = 5 # any value between 1-4 : 1 =100%, 2=50%, 3 = 33%, 4 = 25%, 5 = 20% and so on...
    max_amount = 10000  # Maximum authorized amount
    loss_limit = -50  # Maximum loss limit to terminate the trading in dollar
    buy_percent = 0.004 # percent at which it should buy, currently 0.15% = 0.15/100 = 0.0015
    sell_percent = -0.006 # percent at which it should sell, currently 0.1%
    loss_percent = -0.002 # stop loss if price falls, currently -0.3%
    transaction = 150 # number of maximum transactions
    buy_range = 0.005  # allowed buy upto, currently 1.5%
    sleep_time = 20   # according to candle interval 15 for 5 MINUTE, 30 for 30 MINUTE, 45 for 1 HOUR
    spent_amount = 0
    count = 0
    buy_open = [] # to avoid multiple buy at same candle 
    high ={} 
    symbols_csv_files = []
    while True:
        client.get_deposit_address(asset='USDT')  # USDT or BTC

        try:
            for symbol in list_of_symbols:
                
                # My modifications
                ochl = get_historic_klines(symbol, "May 25, 2019", "now UTC", Client.KLINE_INTERVAL_1DAY)
                open1, close, high1,high_past = ochl[-1][1], ochl[-1][2],ochl[-1][3],ochl[-2][3]
                
                if symbol not in symbols_csv_files:
                    df = pd.DataFrame(ochl, columns=["date", "open", "close", "high", "low", "volume"])
                    
                    f_name = str(symbol) + ".csv"
                    df.to_csv(f_name, columns=["date", "open", "close", "high", "low", "volume"])
                    for i in df.index:
                        df.at[i,'date'] = datetime.fromtimestamp((int(df.at[i,'date']))/1000.0).date()
                    symbols_csv_files.append(symbol)
                    print(symbol)
                    print(df)
                
                if symbol in high:
                    #print(high1)
                    if high1>high_past and open1 not in buy_open:
                        high[symbol] = high1
                    elif high1>high[symbol]:
                        high[symbol]=high1
                symbol = str(symbol)
                if open1 not in buy_open:        
                    if (close  >= (1+buy_percent) * open1) and (symbol not in buy.keys()) and close < (1+buy_range) * open1:
                        if spent_amount <= max_amount:
                            count += 1
                            quantity[symbol] = (max_amount / (quantity_1 * close))
                            quantity1 = quantity[symbol]
                            buy_open.append(open1)
                            high[symbol] = high1
                                      
                            '''
                            client.order_limit_buy(
                                symbol=symbol,
                                quantity=quantity[symbol],
                                price=close  # comment this line to buy at market price
                                )
                            '''
                            spent_amount += close * quantity1
                            buy[symbol] = close
                            print('Bought ' + symbol + ' at ' + str(close))

                            df1 = pd.DataFrame({'Datetime': [datetime.now(tz)], 'Symbol': [symbol], 'Buy/Sell': ['Buy'],
                                        'Quantity': [quantity1], 'Price': [close], 'Profit/loss': [0]})
                            df1['Datetime'] = df1['Datetime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
                            if not os.path.isfile('Binance.csv'):
                                df1.to_csv('Binance.csv', index=False)
                            else:
                                df1.to_csv('Binance.csv', index=False, mode='a', header=False)

                if symbol in buy:
                    #print(close, high[symbol], high1)
                    if (close <= high[symbol] * (1+sell_percent)) or (close  <= (1+loss_percent) * buy[symbol]):
                        '''
                        client.order_limit_sell(
                          symbol=symbol,
                          quantity=quantity[symbol],
                          price=close  # comment this line to sell at market price
                          )
                        '''
                        if close <= high[symbol]*(1+sell_percent):
                            print('high limit' )
                        profit = close - buy[symbol]
                        max_amount += profit
                        quantity1 = quantity[symbol]
                        spent_amount -= quantity1 * buy[symbol]
                        total_profit = profit * quantity1
                        print("SELL " + symbol + " at " + str(close))
                        print("Profit made " + str(total_profit))

                        df2 = pd.DataFrame({'Datetime': [datetime.now(tz)], 'Symbol': [symbol], 'Buy/Sell': ['Sell'],
                                        'Quantity': [quantity1], 'Price': [close], 'Profit/loss': [total_profit]})
                        df2['Datetime'] = df2['Datetime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
                        df2.to_csv('Binance.csv', index=False, mode='a', header=False)

                        loss.append(total_profit)
                        if count >= len(list_of_symbols):
                            loss.pop(0)
                        buy.pop(symbol)  # Removing the sold symbol

                if (loss_limit > sum(loss)) or (count > int(transaction)):
                    print("Quitting....")
                    raise SystemExit

            time.sleep(sleep_time)

        except BinanceAPIException as e:
            print(e,symbol)
        except KeyboardInterrupt:
            print("Total Profit " + str(sum(loss)))
            break

        except SystemExit:
            print("Exit")
            print("Total Profit " + str(sum(loss)))
            break


In [77]:
if __name__ == "__main__":
    print("Inside main")
    Main()

Inside main
XMRUSDT
           date   open  close   high    low       volume
0    2019-05-25  85.32  86.80  87.31  85.02   5615.95591
1    2019-05-26  86.67  91.30  92.39  85.00  12335.57747
2    2019-05-27  91.22  97.43  98.84  91.22  13539.46342
3    2019-05-28  97.31  95.93  98.47  95.28   7523.91455
4    2019-05-29  95.63  93.81  95.91  90.00   9949.82174
..          ...    ...    ...    ...    ...          ...
362  2020-05-21  64.69  61.65  65.14  59.96  65248.26749
363  2020-05-22  61.65  62.89  63.54  61.00  61318.05756
364  2020-05-23  62.89  63.46  65.00  62.57  43248.60649
365  2020-05-24  63.46  60.64  63.71  60.58  54021.43623
366  2020-05-25  60.65  60.98  62.00  60.02  24437.62641

[367 rows x 6 columns]
LINKUSDT
           date    open   close    high     low       volume
0    2019-05-25  1.3314  1.1130  1.3376  1.0961   6245467.50
1    2019-05-26  1.1130  1.0828  1.1824  1.0340   4854605.26
2    2019-05-27  1.0802  1.2119  1.2750  1.0802   7339191.28
3    2019-05-28  1.